<a href="https://colab.research.google.com/github/nalinis07/APT_Ref_Copy_Links/blob/MASTER/AT_Lesson_110_Reference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lesson 110: K-Means Clustering - RFM Analysis

**WARNING:** The reference notebook is meant **ONLY** for a teacher. Please **DO NOT** share it with any student.

The contents of the reference notebook are meant only to prepare a teacher for a class. To conduct the class, use the class copy of the reference notebook. The link and the instructions for the same are provided in the **Notes To The Teacher** section.


|Particulars|Description|
|-|-|
|**Topic**|K-Means Clustering - RFM Analysis|
|||
|**Class Description**|In this class, a student will learn a customer behaviour segmentation technique called RFM.|
|||
|**Class**|C110|
|||
|**Class Time**|55 minutes|
|||
|**Goal**|Removing cancelled orders and null values|
||Understanding RFM analysis|
||Merging DataFrames|
|||
|**Teacher Resources**|Google Account|
||Laptop with internet connectivity|
||Earphones with mic|
|||
|**Student Resources**|Google Account|
||Laptop with internet connectivity|
||Earphones with mic|

---

### Warm-up Quiz

```
TEACHER
I have an exciting quiz question for you! Are you ready to answer this question?

EXPECTED STUDENT RESPONSE
Yes.
```

**Instructions for the Teacher:**
- Please click on the "Quiz Time" button on the bottom right corner of your screen to start the In-Class Quiz. <img src='https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/images/quiz-time.png' width = 150>

- A quiz will be visible to both you and the student. Encourage the student to answer the quiz question.

- The student may choose the wrong option, help the student to think correctly about the question and then answer again.

- After the student selects the correct option, the "End Quiz" button will start appearing on your screen. <img src='https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/images/end-quiz.png' width = 150>  

- Click the "End quiz" button to close the quiz pop-up and continue the class.
  
- Do not spend more than 2 minutes on this quiz.

---

### Teacher-Student Activities

In the previous class, we implemented the K-Means algorithm on a 3D dataset. We created clusters of customers based on their age, annual income, and spending score. We  also started working on a more complex problem statement of Customer Segmentation.

In this class, we will continue working on the same problem statement. We will also learn a strategy for analysing customers based on three factors: Recency, Frequency, and Monetary Value, known as the **RFM** technique.

Let's quickly run the code cells and go through the problem statement covered in the previous lesson and begin this lesson from **Activity 1: Removing the Cancelled Orders**.


---

#### Customer Segmentation Problem Statement


We have a transactional dataset that contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

The company wants to segment its customers and determine marketing strategies according to these segments.

The dataset consists of the following attributes:

- `InvoiceNo`: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.

- `StockCode`: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.

- `Description`: Product (item) name. Nominal.

- `Quantity`: The quantities of each product (item) per transaction. Numeric.

- `InvoiceDate`: Invoice date and time. Numeric, the day and time when each transaction was generated. The date-time format used here is `yyyy-mm-dd hh:mm:ss`.

- `UnitPrice`: Unit price. Numeric, product price per unit in pound sterling, also known as GBP (Great Britain Pound).

- `CustomerID`: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.

- `Country`: Country name. Nominal, the name of the country where each customer resides.



**Dataset Credits:** https://archive.ics.uci.edu/ml/datasets/online+retail

**Citation:** Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.


---

#### Loading the Dataset

Let's import the necessary Python modules (if not imported yet) and read the data from an excel file to create a Pandas DataFrame.



**Dataset Link:** https://s3-student-datasets-bucket.whjr.online/whitehat-ds-datasets/online-retail-customers.xlsx


**Note:** Since the dataset is a Microsoft Excel file, i.e., in the `xlsx` format, we need to use the `read_excel()` function of the Pandas module.

In [ ]:
# Read the dataset and create a Pandas DataFrame.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = "https://s3-student-datasets-bucket.whjr.online/whitehat-ds-datasets/online-retail-customers.xlsx"
df = pd.read_excel(file_path)
df.head()

Now, let's find out the total number of rows and columns, datatypes of columns and missing values (if exist) in the dataset.

In [ ]:
# Get the total number of rows and columns, datatypes of columns and missing values (if exist) in the dataset.
df.info()

There are **541909 rows and 8 columns** and some columns have missing values in the dataset.  We will deal with these missing values in the upcoming section. Let us first deal with cancelled orders.

As per the dataset description,  some of the values in field `InvoiceNo` may start with the letter **'C'**, to indicate cancelled orders. To search for the rows in the dataset where the `InvoiceNo` starts with 'C', we will use **Regular Expressions**.

---

#### The `Series.str.contains()` function

The `Series.str.contains()` function  is used to check if a pattern or regex is contained within a string of a pandas series or not.

For example, let us use  `Series.str.contains()` function to find if a regex `i[a-n]` is present in the strings in the following pandas series:

[`'India', 'China', 'Russia', 'Sweden', 'Syrian_Arab'`]



In [ ]:
# Look for pattern 'ia' in a pandas series.
sr = pd.Series(['India', 'China', 'Russia', 'Sweden', 'Syrian_Arab'])
result = sr.str.contains(pat = 'i[a-n]')

# print the result
print(result)

As we can see in the output, the `Series.str.contains()` function has returned boolean values for each string in the series. It is `True` if the passed regex is present in the string, else `False` is returned.

Now we will use regex to search for cancelled orders in our dataset and will perform other data processing operations.

---

#### Activity 1: Removing the Cancelled Orders

Now that we have learned how to use regex expressions to search for a pattern within a string, we will now utilise them to search for cancelled orders in our dataset.

As per the dataset description, some of the values in field `InvoiceNo` may start with the letter 'C', to indicate cancelled orders. Let us first check the data type of the first row of `InvoiceNo` field.


In [ ]:
# S1.1: Check the data type of 'InvoiceNo' field
type(df['InvoiceNo'][0])

int

The `InvoiceNo` field has integer values but to find the cancelled orders using **RegEx**, it has to be converted to string data type.

Use `DataFrame.astype('str')` function to convert the data type of `InvoiceNo` column to string.

In [ ]:
# S1.2: Convert 'InvoiceNo' field to string and verify whether the data type is converted or not.
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
type(df['InvoiceNo'][0])

str

The values of the `InvoiceNo` column has now been converted into a string. Next, let's search for  the `InvoiceNo` that contains `C` using Regular Expressions.


We will use `Series.str.contains()` function to check whether any `InvoiceNo` column values contains `C`. Recall the syntax of  `Series.str.contains()` function.

**Syntax:** `Series.str.contains(pat, flags = 0, regex = True)`

Pass the following values to the above function:

- `df[df['InvoiceNo']]` as `Series`.
- `pat = C`

-  `flags = re.IGNORECASE`: To include both lowercase and uppercase strings in the search.
- `regex = True` to indicate that the pattern `pat = C`  is a regex.


In [ ]:
# S1.3: Use regex to find 'C' in the 'InvoiceNo' field
import re
df[df['InvoiceNo'].str.contains(pat = 'C', flags = re.IGNORECASE)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In the output, you may observe that there are 9288 orders whose `InvoiceNo` contains 'C'. Thus, there are 9288 cancelled orders.

Let us remove these cancelled orders from total orders. For that, first, check the total number of orders.

In [ ]:
# S1.4: Check total number of orders including cancelled orders.
df['InvoiceNo'].shape[0]

541909

Thus, there are total 541909 orders out of which 9288 orders were cancelled.

Let us remove cancelled orders from the DataFrame which will give us **532621** (541909 - 9288 = 532621) delivered orders.

In [ ]:
# S1.5: Remove cancelled invoices from the dataset
df = df[~(df['InvoiceNo'].str.contains('C', flags = re.IGNORECASE, regex = True))]
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


Thus, we obtained a DataFrame of 532621 rows consisting of delivered orders. Let us now remove the null values from the DataFrame.

---

#### Activity 2: Removing Null Values

Now it's time to check which column has missing or null values. Let us now obtain the total count of null values in each column of `df`.

In [ ]:
# S2.1: Obtain the number of missing or null values in df
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     134697
Country             0
dtype: int64

So, there are null values in the `Description` and `CustomerID` column. Let us obtain the percentage of null values with respect to the total rows in the DataFrame.

In [ ]:
# S2.2: Determine the percentage of null values in each column.
df.isnull().sum() * 100 / df.shape[0]

InvoiceNo       0.000000
StockCode       0.000000
Description     0.272990
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     25.289465
Country         0.000000
dtype: float64

Thus, the `CustomerID` column has around 25% null values and the `Description` column has around 0.27% null values. Let us simply remove these null value rows from the DataFrame.

In [ ]:
# S2.3: Remove the null valued rows.
print(f"Before removing null values:\nNumber of rows = {df.shape[0]}")
df.dropna(inplace = True)
print(f"After removing null values:\nNumber of rows = {df.shape[0]}")

Before removing null values:
Number of rows = 532621
After removing null values:
Number of rows = 397924


Let us reconfirm whether there are still any missing or null values in the DataFrame.

In [ ]:
# S2.4: Again obtain the number of null values in df.
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Our DataFrame is now free from null or missing values. Let us check the data type of `CustomerID` column.

In [ ]:
# S2.5: Check the data type of CustomerID column.
df['CustomerID'].dtype

dtype('float64')

The `CustomerID` column is of float data type. We need to convert it into integer-based categorical column as `CustomerID` cannot be a floating-point value.

In [ ]:
# S2.6: Convert 'CustomerID' field to integer based categorical column.
df['CustomerID'] = df['CustomerID'].astype('int64').astype('category')
df['CustomerID'].dtype

CategoricalDtype(categories=[12346, 12347, 12348, 12349, 12350, 12352, 12353, 12354,
                  12355, 12356,
                  ...
                  18273, 18274, 18276, 18277, 18278, 18280, 18281, 18282,
                  18283, 18287],
                 ordered=False)

Here we can see the `CustomerID` field is now a category based field. Next, proceed with clustering for customer segmentation. For customer segmentation, we will use a well-established approach known as **RFM Analysis**.

---

#### Understanding RFM

RFM stands for Recency, Frequency and Monetary. It is a customer segmentation technique that uses past purchase transactions to divide customers into groups. RFM analysis involves calculating the following three factors:
 - **Recency:**  How recently the customer has made their purchase.
 - **Frequency:**  How often customers have made their purchases.
 - **Monetary:** How much money customers have paid for their purchases.

Let us understand these three factors in more detail.

**1. Recency**

- It indicates how recently the customer has made their purchase.
- It will give you the number of days that have passed since the last purchase made by a customer.
- For example, if `recency = 10` for a particular customer, it means that the last transaction made by that customer was 10 days before.

Consider the following rows from the given dataset:

<img src="https://s3-whjr-v2-prod-bucket.whjr.online/0ff9de93-0dc3-4880-9698-e193c139b47c.PNG"/>


From the above table, it is visible that the customers having `CustomerID` as 12680 and 12462 have recently ordered some items. Thus, the **recency** of purchase can be easily calculated by inspecting `InvoiceDate` and `CustomerID` columns.



**2. Frequency:**

- It indicates how frequently the customers have made their purchases.
- It will give you the total number of transactions made by a customer.
- For example, if `frequency = 2` for a particular customer, it means that the customer has made 2 purchases in total.

Consider the following rows from the given dataset:

<img src="https://s3-whjr-v2-prod-bucket.whjr.online/b0af1160-4d33-4001-b541-76883ba5c8c7.PNG"/>



By looking at `InvoiceNo` and `CustomerID` column, we can say that:
- `CustomerID` 13081 has made 3 transactions.
- `CustomerID` 17850 has made 1 transaction and `CustomerID` 12462 has made 2 transactions.

Thus, the **frequency** of purchase can be easily calculated by inspecting `InvoiceNo` and `CustomerID` columns.

**3. Monetary**

- It indicates how much money customers have paid for their purchases.
- It will give you the total amount spent by a customer.

Consider the following rows from the given dataset:

<img src= 'https://s3-whjr-v2-prod-bucket.whjr.online/b3937de2-b1ab-4e76-9632-b904088e9628.png'/>


The total amount made by a customer in a transaction can be obtained by multiplying the values of `Quantity` and `UnitPrice` columns.


From the above table, we can say that:

- The total amount spent by `CustomerID` 13081 is: $10 \times 1.65 = 16.5$ GBP (Great Britain Pound).

- Total amount spent by `CustomerID` 17850 is: $ (6 \times 2.55) +(6 \times 3.39) = 15.3 + 20.34 = 35.64$ GBP.

Thus, the **monetary** value of the purchase can be easily calculated by inspecting `Quantity`, `UnitPrice` and `CustomerID` columns.

**Summarising RFM:**

Consider customer `A` made 2 transactions in this year: one was 100 days ago with 200 GBP and the other one was 90 days ago with 300 GBP. In this case,
- `recency = 90 days`
- `frequency = 2`
- `monetary = 500 GBP`

**Advantages of RFM model in customer analysis:**

- The more recent the purchase, the more responsive the customer is to promotions.

- The more frequently the customer buys, the more engaged and satisfied they are.

- Monetary value differentiates heavy spenders from low spenders.

Let us now obtain the **RFM** model from our customer segmentation dataset.

---

#### Activity 3: RFM analysis

Let us first check the first 5 rows of this dataset.

In [ ]:
# S3.1: Check the first 5 samples of the DataFrame
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


**Calculating Monetary:**

Monetary value is how much money a customer spends on purchases.  

To calculate the monetary value, we first need to calculate the total purchase value for the customers. This can be obtained by multiplying the values of the `Quantity` column by the `UnitPrice` column.

Let's add a column `TotalPrice` whose values will be the product of `Quantity` and `UnitPrice` column values to the DataFrame.

In [ ]:
# S3.2: Obtain the the total purchase amount for the customers
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


Now we have a new column named: `TotalPrice` which indicates the total amount spent by each customer.

For Monetary, calculate the amount for all the purchases made by every individual customer. For this, first, let us check how many customers do we have in our dataset.

In [ ]:
# S3.3: Obtain the number of unique customers
df['CustomerID'].unique()

[17850, 13047, 12583, 13748, 15100, ..., 13436, 15520, 13298, 14569, 12713]
Length: 4339
Categories (4339, int64): [17850, 13047, 12583, 13748, ..., 15520, 13298, 14569, 12713]

Here we can see that there 4339 categories which imply that our dataset has 4339 unique customers.
The next step is to calculate the total purchase amount spent by these 4339 customers. For this,

1.  Create a DataFrame `monetary_df` consisting of the following two columns:

  1.1. `CustomerID` column.

  1.2. `TotalPrice`  column grouped by unique customers.
  
      - Set the `as_index = False` to reset the index in the new DataFrame.
      - Apply `sum()` function on the grouped DataFrame to get the total amount spent by the customer.

2. Rename the column `TotalPrice` to `Monetary` in `monetary_df` DataFrame.



In [ ]:
# S3.4: Obtain the Monetary information from the DataFrame
monetary_df = df[['CustomerID', 'TotalPrice']].groupby('CustomerID', as_index = False).sum()
monetary_df.rename(columns = {'TotalPrice' : 'Monetary'}, inplace = True)
monetary_df

,CustomerID,Monetary
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40
...,...,...
4334,18280,180.60
4335,18281,80.82
4336,18282,178.05
4337,18283,2094.88


Thus, we have now obtained a DataFrame that tells us the total amount spent by each customer. We can easily differentiate between heavy spenders and low spenders using the `monetary_df` DataFrame.

Let us get the frequency of purchase for respective customers.

**Calculating Frequency:**

Here, we need to count the frequency of purchases made by 4339 unique customers. As already discussed, `CustomerID` and `InvoiceNo` columns are useful to calculate frequency using the steps given below:

1. Create a new DataFrame `frequency_df` consisting of the following columns:
  
  1.1. `CustomerID` column.
  
  1.2. `InvoiceNo` column grouped by unique customers.    
    - Set the `as_index = False` to reset the index in the new DataFrame.
    - Apply `count()` function on the grouped DataFrame to get the total number of invoices for each customer.

2. Rename the column `InvoiceNo` to `Frequency` in `frequency_df` DataFrame.



In [ ]:
# S3.5: Obtain the Frequency information from the DataFrame
frequency_df =  df[['CustomerID', 'InvoiceNo']].groupby('CustomerID', as_index = False).count()
frequency_df.rename(columns = {'InvoiceNo': 'Frequency'}, inplace = True)
frequency_df

,CustomerID,Frequency
0,12346,1
1,12347,182
2,12348,31
3,12349,73
4,12350,17
...,...,...
4334,18280,10
4335,18281,7
4336,18282,12
4337,18283,756


Thus, we obtained a DataFrame that shows the frequency of purchases made by each unique customer.

Let us merge the above 2 DataFrames i.e. `monetary_df` and `frequency_df` into a single DataFrame. Before that, let us first learn what are the different ways of merging DataFrames.

---

#### Activity 4: Merging DataFrames

The `merge()` function of the `pandas` module is used to combine two or more DataFrames, based on a common feature among them. The merging of DataFrames is based on **join** operations. Let us understand the two most commonly used join operations:
1. Inner join
2. Outer join

**1. Inner join:**
- This is the most widely used merging technique.
- It returns a DataFrame with only those rows that have common characteristics.
- An inner join requires each row in the two joined DataFrames to have matching column values.
- This is similar to the intersection of two sets.

<center><img src="https://s3-whjr-v2-prod-bucket.whjr.online/0fe6ee12-b61a-4465-af87-b9144f4c0d9d.png"/></center>

Let us consider a similar problem statement of an online seller in India. Consider a sample DataFrame `products_df` which consists of basic product details.

|Product_ID | Product_name | Category	| Price	|
| -- | -- | -- | -- |
| 1001 |	Watch |	Fashion |	299.0 |
| 1002 |	Bag |	Fashion |	1350.5 |
| 1003 |	Shoes |	Fashion |	2999.0 |
| 1004 | Smartphone |	Electronics |	14999.0 |
| 1005 | Books |	Study |	145.0 |
| 1006 |	Cheese |	Grocery |	110.0 |
| 1007 |	Laptop |	Electronics |	79999.0 |

Consider another sample DataFrame `customer_df` of the same company keeping track of the customers details and their purchase.

| Customer_ID |	Name |	Age |	Product_ID |	Purchased_Product |
| -- | -- | -- | -- | -- |
|1	| Priyanshu	| 20	| 1001	| Watch	|
|2	| Rohit	| 25	| 0	| NA	|
|3	| Apeksha	| 15 | 1006 |	Oil |
|4	| Rohan	| 10	| 0 | NA |
|5	| Karan	| 30	| 1003 |	Shoes |
|6	| Divya	| 65	| 1004 |	Smartphone |
|7	| Abhinav	| 35	| 0 |	NA |
|8	| Isha	| 18	| 0 |	NA |
|9	| Vivek	| 23	| 1007|	Laptop |

Let's say the company needs information on all the products sold online as well as the details of customers who purchased those products. This information can be obtained by merging both the DataFrames based on the common column `Product_ID` using inner join.

**Syntax for inner join:**

`pd.merge(df1, df2, on='common_column', how='inner')`

Where,
 `df1` and `df2` are the two DataFrames to be merged.

The inner join will return only those rows from both the DataFrames that have common `ProductID`.

Let us see how this works. First, create both the DataFrames and apply inner join to merge both the DataFrames.


In [ ]:
# T4.1: Create 'products_df' DataFrame.

products_df = pd.DataFrame({
    'Product_ID':[1001,1002,1003,1004,1005,1006,1007],
    'Product_name':['Watch','Bag','Shoes','Smartphone','Books','Cheese','Laptop'],
    'Category':['Fashion','Fashion','Fashion','Electronics','Study','Grocery','Electronics'],
    'Price':[299.0,1350.50,2999.0,14999.0,145.0,110.0,79999.0],
    })

products_df

,Product_ID,Product_name,Category,Price
0,1001,Watch,Fashion,299.0
1,1002,Bag,Fashion,1350.5
2,1003,Shoes,Fashion,2999.0
3,1004,Smartphone,Electronics,14999.0
4,1005,Books,Study,145.0
5,1006,Cheese,Grocery,110.0
6,1007,Laptop,Electronics,79999.0


Create a new DataFrame to which we will apply inner join.

In [ ]:
# T4.2: Create 'customer_df' DataFrame.

customer_df = pd.DataFrame({
    'Customer_ID':[1,2,3,4,5,6,7,8,9],
    'name':['Priyanshu','Rohit','Apeksha','Rohan','Karan','Divya','Abhinav','Isha','Vivek'],
    'age':[20,25,15,10,30,65,35,18,23],
    'Product_ID':[1001,0,1006,0,1003,1004,0,0,1007],
    'Purchased_Product':['Watch','NA','Oil','NA','Shoes','Smartphone','NA','NA','Laptop'],
   })

customer_df

,Customer_ID,name,age,Product_ID,Purchased_Product
0,1,Priyanshu,20,1001,Watch
1,2,Rohit,25,0,NA
2,3,Apeksha,15,1006,Oil
3,4,Rohan,10,0,NA
4,5,Karan,30,1003,Shoes
5,6,Divya,65,1004,Smartphone
6,7,Abhinav,35,0,NA
7,8,Isha,18,0,NA
8,9,Vivek,23,1007,Laptop


Let's now merge the `products_df` and `customer_df` DataFrames using the inner join approach.

In [ ]:
# S4.1: Use inner join to merge both DataFrames

pd.merge(products_df, customer_df, on='Product_ID', how = 'inner')

,Product_ID,Product_name,Category,Price,Customer_ID,name,age,Purchased_Product
0,1001,Watch,Fashion,299.0,1,Priyanshu,20,Watch
1,1003,Shoes,Fashion,2999.0,5,Karan,30,Shoes
2,1004,Smartphone,Electronics,14999.0,6,Divya,65,Smartphone
3,1006,Cheese,Grocery,110.0,3,Apeksha,15,Oil
4,1007,Laptop,Electronics,79999.0,9,Vivek,23,Laptop


From the output, you may observe that only rows having matching `ProductID` are obtained from both the DataFrames.

But what if we need to combine both DataFrames such that we can find all the products that are not sold and all the customers who didn’t purchase anything from us. In such a case, we will use **outer join** operation.

**Outer Join:**
- An outer join returns a set of records (or rows) that include what an inner join would return along with other rows for which no corresponding match is found in the other table.
- The fields where matching data is missing, nulls are produced.

<center><img src="https://s3-whjr-v2-prod-bucket.whjr.online/260fc528-a6ba-47cf-90b1-2fb207e5377d.png"/></center>

  **Syntax for outer join:**

  `pd.merge(df1, df2, on='common_column', how='outer')`

  Where,
   `df1` and `df2` are the two DataFrames to be merged.

Let us perform outer join operation on both the DataFrames and observe the results.

In [ ]:
# S4.2: Use outer join to merge both DataFrames

pd.merge(products_df, customer_df, on='Product_ID', how = 'outer')

,Product_ID,Product_name,Category,Price,Customer_ID,name,age,Purchased_Product
0,1001,Watch,Fashion,299.0,1.0,Priyanshu,20.0,Watch
1,1002,Bag,Fashion,1350.5,NaN,NaN,NaN,NaN
2,1003,Shoes,Fashion,2999.0,5.0,Karan,30.0,Shoes
3,1004,Smartphone,Electronics,14999.0,6.0,Divya,65.0,Smartphone
4,1005,Books,Study,145.0,NaN,NaN,NaN,NaN
5,1006,Cheese,Grocery,110.0,3.0,Apeksha,15.0,Oil
6,1007,Laptop,Electronics,79999.0,9.0,Vivek,23.0,Laptop
7,0,NaN,NaN,NaN,2.0,Rohit,25.0,NA
8,0,NaN,NaN,NaN,4.0,Rohan,10.0,NA
9,0,NaN,NaN,NaN,7.0,Abhinav,35.0,NA


From the above output, you may observe that the `merge()` function returned `NaN` for every column of the DataFrame that lacks a matching row.

Let us now use the `merge()` function to merge the two DataFrames `monetary_df` and `frequency_df` using inner join. Store the merged DataFrame in the `rfm_df` variable.


In [ ]:
# S4.3: Merge 'monetary_df' and 'frequency_df' DataFrames.
rfm_df = pd.merge(monetary_df, frequency_df, on='CustomerID', how = 'inner')
rfm_df.head()

,CustomerID,Monetary,Frequency
0,12346,77183.60,1
1,12347,4310.00,182
2,12348,1797.24,31
3,12349,1757.55,73
4,12350,334.40,17


Now that we have obtained monetary and frequency values for RFM analysis, let us proceed with calculating the last factor i.e  `recency` for our dataset.

---

#### Activity 5: Calculating Recency

For recency, we need to calculate the number of days between the present date and the date of the last purchase made by each customer. As already discussed, `CustomerID` and `InvoiceDate` columns can be used to obtain  the date of last purchase made by a customer. For this,

1. Create a DataFrame `recency_df` consisting of the following columns:
  
  1.1. `CustomerID` column.
  
  1.2. `InvoiceDate` column grouped by unique customers.
      - Set the `as_index = False` to reset the index in the new DataFrame.
      - Apply `max()` function to get the latest purchase date for each customer.

2. Rename the column `InvoiceDate` to `LastPurchaseDate` in `recency_df` DataFrame.

In [ ]:
# S5.1: Obtain the last purchase date for each customer
recency_df = df[['CustomerID', 'InvoiceDate']].groupby('CustomerID', as_index = False).max()
recency_df.rename(columns = {'InvoiceDate': 'LastPurchaseDate'}, inplace = True)
recency_df

,CustomerID,LastPurchaseDate
0,12346,2011-01-18 10:01:00
1,12347,2011-12-07 15:52:00
2,12348,2011-09-25 13:13:00
3,12349,2011-11-21 09:51:00
4,12350,2011-02-02 16:01:00
...,...,...
4334,18280,2011-03-07 09:52:00
4335,18281,2011-06-12 10:53:00
4336,18282,2011-12-02 11:43:00
4337,18283,2011-12-06 12:02:00


Now, we have obtained a DataFrame that shows the last purchase date of each customer. But for recency, we need the number of days between the last purchase date and the present date (or a reference date).

First, find out the last invoice date in the dataset from the `InvoiceDate` column.

In [ ]:
# S5.2: Obtain the last invoice date in the dataset.
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

Since the last invoice date is `2011–12–09`, we will consider `2011–12–10` as the present date to calculate recency.

**Note:** You can also consider `2011–12–09` as the present date to calculate recency. However, in that case,  we will obtain recency as `0` for purchases made on `2011–12–09` date.

So let us first add 1 day to the last invoice date to obtain the present date (i.e. `2011–12–10`). To do so, use the `Timedelta()` function of `pandas` module as follows:

`pd.Timedelta("1 day")`

**Note:** You can use `help(pd.Timedelta)` function to learn more about the syntax of `Timedelta()` function.

In [ ]:
# S5.3: Obtain the present date i.e LastPurchaseDate + 1 day
present_date = df['InvoiceDate'].max() + pd.Timedelta("1 day")
present_date

Timestamp('2011-12-10 12:50:00')

Now, we have both the dates i.e. the last purchase date of each customer (`recency_df['LastPurchaseDate']`) and the present date (`present_date`). We can now easily calculate the number of days that have passed since the customer's  last purchase.


In [ ]:
# S5.4: Obtain the days since last purchase made by a customer
days_last_purchase = present_date - recency_df['LastPurchaseDate']
days_last_purchase

0      326 days 02:49:00
1        2 days 20:58:00
2       75 days 23:37:00
3       19 days 02:59:00
4      310 days 20:49:00
              ...       
4334   278 days 02:58:00
4335   181 days 01:57:00
4336     8 days 01:07:00
4337     4 days 00:48:00
4338    43 days 03:21:00
Name: LastPurchaseDate, Length: 4339, dtype: timedelta64[ns]

We obtained the `timedelta64` object after subtracting the last purchase date from the present date for each customer. (Here, `64` indicates 64-bit integers.)

However, for recency we are interested only in the number of days that have elapsed since the last order. To extract days, use `dt.days` attribute with the above pandas series.

In [ ]:
# S5.5: Extract days from datetime using 'dt.days' attribute
recency_days = days_last_purchase.dt.days
recency_days

0       326
1         2
2        75
3        19
4       310
       ... 
4334    278
4335    181
4336      8
4337      4
4338     43
Name: LastPurchaseDate, Length: 4339, dtype: int64

Thus, we obtained recency for 4339 customers. Let's add these days as a column `Recency` to the `rfm_df` DataFrame.

In [ ]:
# S5.6: Add 'recency_days' as column to the merged DataFrame 'rfm_df'.
rfm_df['Recency'] = recency_days
rfm_df

,CustomerID,Monetary,Frequency,Recency
0,12346,77183.60,1,326
1,12347,4310.00,182,2
2,12348,1797.24,31,75
3,12349,1757.55,73,19
4,12350,334.40,17,310
...,...,...,...,...
4334,18280,180.60,10,278
4335,18281,80.82,7,181
4336,18282,178.05,12,8
4337,18283,2094.88,756,4


We now have a DataFrame for **RFM** analysis consisting of the necessary fields to carry out the customer segmentation.

In the next class, we will analyse the RFM  DataFrame and prepare it for K-Means clustering.

---

#### Activities

**Teacher Activities**

1. K-Means Clustering - RFM Analysis (Class Copy)

   Link on Panel

2. K-Means Clustering - RFM Analysis (Reference)

   Link on Panel

---